In [1]:
import numpy as np
import pandas as pd
import pickle
import xgboost
import os
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel
from config import path_prefix_results
import warnings

warnings.filterwarnings('ignore')


## Create latex template for leg, loin, belly, shoulder, body results

In [2]:
#latex output setup
latex_template = r'''\documentclass[a4paper]{{article}}
\usepackage{{booktabs}}
\usepackage{{pdflscape}}
\begin{{document}}
\thispagestyle{{empty}}
\begin{{landscape}}
{}
\end{{landscape}}
\end{{document}}
'''

path_body_r2_latex= os.path.join(path_prefix_results, 'body_r2.tex')
path_body_rmse_latex= os.path.join(path_prefix_results, 'body_rmse.tex')
path_body_ttest_latex= os.path.join(path_prefix_results, 'body_ttest.tex')
path_body_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'body_fs_ttest_diff.tex')
path_body_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'body_fs_ttest_diff_cross.tex')

In [3]:
def test_diff_cross(df, row='multi-atlas'):
    tmp= pd.DataFrame(index=df.columns, columns=df.columns)
    for i in df.columns:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[row, i]), np.abs(df.loc[row, j]))[1]
    return tmp

def test_diff(df, col='multi-atlas'):
    tmp= pd.DataFrame(index=df.index, columns=df.columns)
    for i in df.index:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[i, j]), np.abs(df.loc[col, j]))[1]
    tmp= tmp.drop(col, axis='rows')
    return tmp

def test_compare(df0, df1):
    tmp= pd.DataFrame(index=df0.index, columns=df0.columns)
    for i in df0.index:
        for j in df0.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df0.loc[i, j]), np.abs(df1.loc[i, j]))[1]
    return tmp

## Load data 

In [4]:
data_fs= pickle.load(open(os.path.join(path_prefix_results,'results.pickle'), 'rb'))
data_no_fs= pickle.load(open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'results/results.pickle'

In [ ]:
body_fs= data_fs[data_fs['dataset'] == 'body']

body_no_fs= data_no_fs[data_no_fs['dataset'] == 'body']

In [ ]:
rename_dict_regr= {'KNNR_Objective': 'kNN', 'LinearRegression_Objective': 'linear', 'PLSRegression_Objective': 'PLS', 'RidgeRegression_Objective': 'ridge', 'LassoRegression_Objective': 'lasso'}
#rename_dict_type= {'203a': 'atlas 1', '203k': 'atlas 2', '204f': 'atlas 3', '206k': 'atlas 4', '208f': 'atlas 5', 'all': 'multi-atlas', 'mean_mask': 'mean'}
#row_order= ['atlas 1', 'atlas 2', 'atlas 3', 'atlas 4', 'atlas 5', 'mean', 'multi-atlas']
rename_dict_type= sorted(leg_fs['type'].unique())
rename_dict_type= dict(map(lambda x, i: (x, f"atlas {i+1}"), rename_dict_type,  range(len(rename_dict_type))))
rename_dict_type['all']= "multi-atlas"
rename_dict_type['mean_mask'] = "mean"
print(rename_dict_type)
row_order= list(rename_dict_type.values())
col_order= ['linear', 'PLS', 'lasso', 'ridge', 'kNN']

## body $r^2$ results

In [ ]:
body_fs_results= body_fs.pivot(index='type', columns='model', values='r2_1')
body_fs_results= body_fs_results.rename(rename_dict_regr, axis='columns')
body_fs_results= body_fs_results.rename(rename_dict_type)
body_fs_results= body_fs_results.apply(lambda x: np.round(x, 4))
body_fs_results= body_fs_results.loc[row_order, col_order]

body_no_fs_results= body_no_fs.pivot(index='type', columns='model', values='r2_1')
body_no_fs_results= body_no_fs_results.rename(rename_dict_regr, axis='columns')
body_no_fs_results= body_no_fs_results.rename(rename_dict_type)
body_no_fs_results= body_no_fs_results.apply(lambda x: np.round(x, 4))
body_no_fs_results= body_no_fs_results.loc[row_order, col_order]

body_all= pd.concat([body_fs_results, body_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
body_all.columns= columns

In [ ]:
body_all_r2_latex= body_all.to_latex()
print(body_all_r2_latex)

#save results to tex file
with open(path_body_r2_latex, 'w') as f:
    f.write(latex_template.format(body_all_r2_latex))

## body RMSE results

In [ ]:
body_fs_results_rmse= body_fs.pivot(index='type', columns='model', values='rmse_1')
body_fs_results_rmse= body_fs_results_rmse.rename(rename_dict_regr, axis='columns')
body_fs_results_rmse= body_fs_results_rmse.rename(rename_dict_type)
body_fs_results_rmse= body_fs_results_rmse.apply(lambda x: np.round(x, 4))
body_fs_results_rmse= body_fs_results_rmse.loc[row_order, col_order]

body_no_fs_results_rmse= body_no_fs.pivot(index='type', columns='model', values='rmse_1')
body_no_fs_results_rmse= body_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
body_no_fs_results_rmse= body_no_fs_results_rmse.rename(rename_dict_type)
body_no_fs_results_rmse= body_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
body_no_fs_results_rmse= body_no_fs_results_rmse.loc[row_order, col_order]

body_all_rmse= pd.concat([body_fs_results_rmse, body_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
body_all_rmse.columns= columns

In [ ]:
body_all_rmse_latex= body_all_rmse.to_latex()
print(body_all_rmse_latex)

#save results to tex file
with open(path_body_rmse_latex, 'w') as f:
    f.write(latex_template.format(body_all_rmse_latex))

## t-test body

In [ ]:
body_fs['diff']= (body_fs['y_pred_1'] - body_fs['y_test_1'])**2
body_fs_test= body_fs.pivot(index='type', columns='model', values='diff')
body_fs_test= body_fs_test.rename(rename_dict_regr, axis='columns')
body_fs_test= body_fs_test.rename(rename_dict_type)
body_fs_test= body_fs_test.loc[row_order, col_order]

body_no_fs['diff']= (body_no_fs['y_pred_1'] - body_no_fs['y_test_1'])**2
body_no_fs_test= body_no_fs.pivot(index='type', columns='model', values='diff')
body_no_fs_test= body_no_fs_test.rename(rename_dict_regr, axis='columns')
body_no_fs_test= body_no_fs_test.rename(rename_dict_type)
body_no_fs_test= body_no_fs_test.loc[row_order, col_order]

In [ ]:
body_ttest_latex= test_compare(body_fs_test, body_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(body_ttest_latex)

#save results to tex file
with open(path_body_ttest_latex, 'w') as f:
    f.write(latex_template.format(body_ttest_latex))

In [ ]:
body_fs_ttest_diff_latex= test_diff(body_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(body_fs_ttest_diff_latex)

#save results to tex file
with open(path_body_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(body_fs_ttest_diff_latex))

In [ ]:
body_fs_ttest_diff_cross_latex= test_diff_cross(body_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(body_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_body_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(body_fs_ttest_diff_cross_latex))

### Convert tex files to pdfs

In [ ]:
import shutil
import glob

pdflatex_path= shutil.which("pdflatex")
if path_prefix_results != os.path.basename(os.getcwd()):
    os.chdir(path_prefix_results)
texs= glob.glob("*tex")
temp = "tmp"
os.makedirs(temp, exist_ok=True)

In [ ]:
for tex in texs:
    print(f"Converting {tex}...")
    cmd= f"{pdflatex_path} {tex} >> /dev/null"
    os.system(cmd)
    for i in glob.glob("*.aux") + glob.glob("*.log"):
        shutil.move(os.path.abspath(i), os.path.join(os.path.abspath(temp),i))